In [ ]:
import math
import numpy as np
import random
from itertools import chain, combinations
from copy import deepcopy

epsilon = float(input("Enter the value of epsilon: "))
delta = float(input("Enter the value of delta: "))

agents = {1,2,3,4}
n = len(agents)

R = agents
Partition = []
S = {}
wsamples = []
w2samples = []
z=set()
sampledCoalitions = []
snew = []
#U = set()

m = (2*pow(n,4) + 2*pow(n,3))*math.ceil(((1/epsilon)*math.log(2*pow(n,3)/delta)))
w = math.ceil(2*pow(n,2)*(1/epsilon)*math.log(2*pow(n,3)/delta))
print(m)
print(w)

agentPreferences= {
      1: [{1,2}, {1,2,3}, {1,2,4}, {1,2,3,4}, {1,3}, {1,3,4}, {1,4}, {1}],
      2: [{2,3}, {2,3,4}, {1,2,3}, {1,2,3,4}, {2,4}, {1,2,4}, {1,2}, {2}],
      3: [{1,3}, {1,2,3}, {1,3,4}, {1,2,3,4}, {2,3}, {2,3,4}, {3,4}, {3}],
      4: [{3,4}, {1,3,4}, {2,3,4}, {1,2,3,4}, {2,4}, {1,2,4}, {1,4}, {4}],
      }

agentUtilities = {
      1: [9.5, 9.4, 9.4, 9.3, 7.5, 7.4, 6.5, 5],
      2: [9.5, 9.4, 9.4, 9.3, 7.5, 7.4, 6.5, 5],
      3: [9.5, 9.4, 9.4, 9.3, 7.5, 7.4, 6.5, 5],
      4: [9.5, 9.4, 9.4, 9.3, 7.5, 7.4, 6.5, 5]
}

def all_subsets(AA):
    
    def powerset(A):
        length = len(A)
        return {
        frozenset({e for e, b in zip(A, f'{i:{length}b}') if b == '1'})
        for i in range(2 ** length)}

    s = set()
    s = powerset(AA)
    l = list(s)
    l1 = [set(x) for x in l]
    #print(list(sub_sets))
    return l1

sampledCoalitionsarray = np.random.choice(all_subsets(agents), m)
sampledCoalitions = sampledCoalitionsarray.tolist()
#print(sampledCoalitions)
def getAgentUtilities(coalition, i):
    l = agentPreferences[i]
    f=0
    for k in range(len(l)):
        if (l[k] == coalition):
            f=1
            break;
    if f==1:
        l1 = agentUtilities[i]
        util = l1[k]
    else:
        util = 0
    return util
    
def getUtilities(coalition):
    utility = []
    for i in coalition:
        l = agentPreferences[i]
        for k in range(len(l)):
            if (l[k] == coalition):
                a = k
        l2 = agentUtilities[i]
        utility.append(l2[a])
    return utility
        
#print(getUtilities({1,2,4}))  
#N = list()
samples = dict()

#for i in sampledCoalitions:
 #   samples[str(i)] = getUtilities(i)

#print(samples)

def getApproximateChoice(R):
    global snew
    global sampledCoalitions
#     print(len(snew))
#     print(len(sampledCoalitions))
#     print(R)
    choiceSets = dict((i, set()) for i in R)
 #print(choiceSets)
    U = set()
    for x in snew:
        U=U.union(x)
   # print(U)
    #print(len(sampledCoalitions))
    for i in R:
        if i not in U:
            choiceSets[i] = {i}
        else:
            max = 0
            for T in snew:
                l2 = getAgentUtilities(T, i)
                if l2>max:
                    max = l2
                    arg = T
            choiceSets[i] = arg
        storeintersection = set()
        for T in snew:
            if (math.floor(getAgentUtilities(T,i)) == math.floor(getAgentUtilities(choiceSets[i], i))):
                if storeintersection == set():
                    storeintersection = T
                else:
                    storeintersection = storeintersection.intersection(T)
            if storeintersection != set():
                choiceSets[i] = storeintersection
                
    for j in range(1, (len(R)+1)):
        
        w2samplesarray = np.random.choice(sampledCoalitions,w)
        #print(w2samplesarray)
        w2samples = w2samplesarray.tolist()
        
       # print(len(w2samples))
        for i in w2samples:
            sampledCoalitions.remove(i)
       # print(len(sampledCoalitions))
        #s__ = sampledCoalitions
        #print(sampledCoalitions)
        for i in R:
            #storeintersection2 = choiceSets[i]
            storeintersection3 = set()
            for T in sampledCoalitions:
                if (math.floor(getAgentUtilities(T,i)) == math.floor(getAgentUtilities(choiceSets[i], i))):
                    if storeintersection3 == set():
                        storeintersection3 = T
                    else:
                        storeintersection3 = storeintersection3.intersection(T)
                if storeintersection3 != set():
                    choiceSets[i] = choiceSets[i].intersection(storeintersection3)
    return choiceSets
                    
                    
def getChoiceCovering(c1, N, F):
    global z
    if(N<=2):
        for x in c1:
            #print(b[i])
            z=z.union(F[x])
        return z
    else:
        for x in getChoiceCovering(c1, N-1, F):
            z=z.union(F[x])
        return z
                      
while R!=set():
    global S
    global sampledCoalitions
    global snew
    global z
    global wsamples
    wsamplesarray = np.random.choice(sampledCoalitions,w)
    wsamples = wsamplesarray.tolist()
    #print(len(wsamples))
    for i in wsamples:
        sampledCoalitions.remove(i)
    #print(len(sampledCoalitions))
    
    snew = deepcopy(sampledCoalitions)
#     print(len(sampledCoalitions))
#     print(len(snew))
   
    sub_sets_now = all_subsets(R)
    print(sub_sets_now)
    
    snew = [x for x in snew if x in sub_sets_now]
    
    #print(len(snew))
    #print(snew)
    #print(len(sampledCoalitions))
    small = 10;
    #print(small)
    f = getApproximateChoice(R)
    print(f)
    for i in R:
#         print(f[i])
#         print(len(R))
#         print(f)
        print(getChoiceCovering(f[i], len(R), f))
        if(len(getChoiceCovering(f[i], len(R), f))<small):
            small = len(getChoiceCovering(f[i], len(R), f))
            #print(getChoiceCovering(f[i], len(R), f))
            index = i
        z = set()
    #print(small)
#     print(getChoiceCovering(f[index], len(R), f))
    S = getChoiceCovering(f[index], len(R), f)
    z = set()
    print(S)
    Partition.append(S)
    print(Partition)
    R = R - S
    print(R)
    
print(Partition)
            
    
#print(getChoiceCovering({4}, 1, {4:{4}}))

  